# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import pickle
import nltk
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import word_tokenize 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download(['wordnet', 'punkt', 'stopwords'])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName',engine)
X = df['message']
Y = df.drop(['id','message','original','genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Function:generates the model classifications
    Input: text
    Output: the classification
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    text_words = word_tokenize(text)   

    stop_words = stopwords.words("english")
    words = [i for i in text_words if i not in stop_words]
    
    clean_tokens = [WordNetLemmatizer().lemmatize(x) for x in text_words]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# train
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3, random_state = 60)

In [6]:
# Fit
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def classification(model, X_test, y_test):

    ''' generates the model classifications
    Input: X_test and y_test
    Output: the classification '''

    y_pred = model.predict(X_test)
    for j, column in enumerate(y_test):
        print(column)
        print(classification_report(y_test[column], y_pred[:, j]))
        

In [8]:
classification(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.63      0.38      0.47      1781
          1       0.83      0.93      0.88      6029
          2       0.54      0.36      0.43        55

avg / total       0.78      0.80      0.78      7865

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6475
          1       0.84      0.41      0.55      1390

avg / total       0.88      0.88      0.86      7865

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7824
          1       0.00      0.00      0.00        41

avg / total       0.99      0.99      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      4583
          1       0.74      0.52      0.62      3282

avg / total       0.73      0.73      0.72      7865

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:

parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10, 20], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)
classification(cv, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.70      0.32      0.44      1781
          1       0.82      0.96      0.89      6029
          2       0.58      0.40      0.47        55

avg / total       0.79      0.81      0.78      7865

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      6475
          1       0.89      0.36      0.51      1390

avg / total       0.88      0.88      0.86      7865

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7824
          1       0.00      0.00      0.00        41

avg / total       0.99      0.99      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.74      0.87      0.80      4583
          1       0.76      0.57      0.65      3282

avg / total       0.75      0.74      0.74      7865

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline_2_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('best', TruncatedSVD()),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])


pipeline_2_ada.fit(X_train, y_train)
classification(pipeline_2_ada, X_test, y_test)

parameters_2_ada =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100], 
              'clf__estimator__min_samples_split': [2, 4]}

cv_2_ada = GridSearchCV(pipeline_2_ada, param_grid=parameters_2_ada)
cv_2_ada

related
             precision    recall  f1-score   support

          0       0.17      0.03      0.05      1781
          1       0.76      0.96      0.85      6029
          2       1.00      0.02      0.04        55

avg / total       0.63      0.74      0.66      7865

request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      6475
          1       0.00      0.00      0.00      1390

avg / total       0.68      0.82      0.74      7865

offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      7824
          1       0.00      0.00      0.00        41

avg / total       0.99      0.99      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.74      4583
          1       0.60      0.00      0.01      3282

avg / total       0.59      0.58      0.43      7865

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 9. Export your model as a pickle file

In [12]:
 # Pickle file
file_name = 'model_classifier.pkl'

with open (file_name, 'wb') as f:
    pickle.dump(cv_2_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.